# Creación de grafos
Funcionalidades para transformar los archivos ".csv" dados en grafos guardados en archivos con la extensión ".gexf".

Dos tipos de modos: 
* MODE = "user": red de usuarios unidos si han usado uno (o más) hashtags
* MODE = "hashtag": red de hashtags unidos si han sido usados por uno (o más) usuarios

In [1]:
import networkx as nx
import pandas as pd
from tqdm import tqdm
import os

In [2]:
data_folder = "data/"
graphs_folder_base = "graphs/nodes_"

In [22]:
def create_bipartite_graph(df, graphs_folder, MANIFESTACION):
    df_h = df["hour"].unique()
    print("Creando redes bipartitas, manifestación seleccionada:", MANIFESTACION, "número de horas: ", len(df_h))
    G = nx.Graph()
    for hour in tqdm(df_h):
        df_hour = df[(df["hour"] == hour)]
        G = nx.from_pandas_edgelist(df_hour, source="user", target="hashtag", edge_attr="weight")
        nx.write_gexf(G, graphs_folder + str(hour) + ".gexf")

In [44]:
def create_graphs(node_criteria, edge_criteria, df, graphs_folder, filtered=False, thresh_filt=5):
    df_h = df["hour"].unique()
    print("Creando redes de", node_criteria, "unidos si comparten uno o más", edge_criteria, ", manifestación seleccionada:", MANIFESTACION, "número de horas: ", len(df_h))
    for hour in tqdm(df_h):
        G = nx.Graph()
        df_hour = df[(df["hour"] == hour)]
        df_nodes = df_hour[node_criteria].unique()
        G.add_nodes_from(df_nodes)
        for node in df_nodes:
            # Seleccionamos las filas del dataframe con el usuario/hashtag sobre el que iteramos
            df_node_edge = df_hour.loc[df_hour[node_criteria] == node]
            # Seleccionamos tantos hashtags/usuarios como haya que haya compartido el usuario/hasthag respectivamente
            df_node_edge = df_node_edge[edge_criteria]
            for edge in df_node_edge:
                df_edge = df_hour.loc[df_hour[edge_criteria] == edge]
                df_edge = df_edge[node_criteria]
                for nd in df_edge:
                    if nd != node:
                        if G.has_edge(node, nd):
                            G[node][nd]["weight"] += 1
                        else:
                            G.add_edge(node, nd, weight = 1)
        
        # Finalmente dividimos entre dos todos los pesos de las aristas, pues están contados dos veces (uno por cada nodo)
        for edge in G.edges():
            old_weight = G.edges[edge]["weight"]
            nx.set_edge_attributes(G, {edge: {"weight": old_weight/2}})
        if filtered:
            H = nx.Graph()

            # Añadir nodos del grafo original al nuevo grafo
            H.add_nodes_from(G.nodes())

            # Añadir aristas que cumplen con el umbral de peso
            for u, v, data in G.edges(data=True):
                if data['weight'] >= thresh_filt:
                    H.add_edge(u, v, **data)
            nx.write_gexf(H, graphs_folder + "filtered/" + str(thresh_filt) + '/' + str(hour) + ".gexf")
        else:
            nx.write_gexf(G, graphs_folder + str(hour) + ".gexf")
        #print("Creado grafo con", G.number_of_nodes(), "nodos y", G.number_of_edges(), "aristas.")

## MANIESTACIÓN 9N

In [37]:
# Seleccionamos el fichero correspondie9nnte a la manifestación social de la que queremos sacar la red
MANIFESTACION = "nat"
file_selected = data_folder + MANIFESTACION + '.txt'
df_9n = pd.read_csv(file_selected, sep=' ')

### MODE HASHTAGS AS NODES

In [41]:
graphs_folder = graphs_folder_base  + "hashtag" + '/' + MANIFESTACION + '/'
create_graphs("hashtag", "user", df_9n, graphs_folder, filtered=False, thresh_filt=5)

Creando redes de hashtag unidos si comparten uno o más user , manifestación seleccionada: nat número de horas:  260


100%|██████████| 260/260 [00:55<00:00,  4.73it/s]


### MODE USERS AS NODES

In [39]:
graphs_folder = graphs_folder_base  + "user" + '/' + MANIFESTACION + '/'
create_graphs("user", "hashtag", df_9n, graphs_folder)

Creando redes de user unidos si comparten uno o más hashtag , manifestación seleccionada: nat número de horas:  260


100%|██████████| 260/260 [00:43<00:00,  5.92it/s]


### MODE BIPARTITE GRAPH

In [25]:
graphs_folder = graphs_folder_base + "bipartite/" + MANIFESTACION + '/'
create_bipartite_graph(df_9n, graphs_folder, "9n")

Creando redes bipartitas, manifestación seleccionada: 9n número de horas:  97


100%|██████████| 97/97 [00:05<00:00, 18.21it/s]


## MANIFESTACION NAT

In [42]:
# Seleccionamos el fichero correspondiente a la manifestación social de la que queremos sacar la red
MANIFESTACION = "nat"
file_selected = data_folder + MANIFESTACION + '.txt'
df_nat = pd.read_csv(file_selected, sep=' ')

### MODE HASHTAGS AS NODES

In [46]:
graphs_folder = graphs_folder_base  + "hashtag" + '/' + MANIFESTACION + '/'
create_graphs("hashtag", "user", df_nat, graphs_folder, filtered=True, thresh_filt=2)

Creando redes de hashtag unidos si comparten uno o más user , manifestación seleccionada: nat número de horas:  260


100%|██████████| 260/260 [01:14<00:00,  3.47it/s]


### MODE USERS AS NODES

In [35]:
graphs_folder = graphs_folder_base  + "user" + '/' + MANIFESTACION + '/'
create_graphs("user", "hashtag", df_nat, graphs_folder)

Creando redes de user unidos si comparten uno o más hashtag , manifestación seleccionada: nat número de horas:  260


100%|██████████| 260/260 [00:36<00:00,  7.21it/s]


### MODE BIPARTITE GRAPH

In [30]:
graphs_folder = graphs_folder_base + "bipartite/" + MANIFESTACION + '/'
create_bipartite_graph(df_nat, graphs_folder, "nat")

Creando redes bipartitas, manifestación seleccionada: nat número de horas:  260


100%|██████████| 260/260 [00:04<00:00, 54.18it/s]
